# Imports

In [175]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import wrangle

from os.path import exists

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from importlib import reload

import warnings
#warnings.filterwarnings("ignore")


# Acquire

In [96]:
reload(wrangle)

<module 'wrangle' from '/Users/sinao/coding_projects/oct_2022_unemployment_project/wrangle.py'>

In [97]:
df = pd.read_csv('oct22pub.csv')

In [98]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126649 entries, 0 to 126648
Columns: 388 entries, hrhhid2 to pternh2
dtypes: float64(337), int64(51)
memory usage: 374.9 MB


In [99]:
# list to hold columns for masking. 

In [100]:
columns_to_keep = [ 'hehousut',
                    'hrnumhou', 
                    'hefaminc',
                    'hrhtype',
                    'hubus',
                    'gediv',
                    'gestfips',
                    'gtmetsta',
                    'gtcbsasz',
                    'prtage',
                    'pemaritl',
                    'pesex',
                    'peafever',
                    'peeduca',
                    'ptdtrace', 
                    'pehspnon',
                    'penatvty',
                    'pemntvty',
                    'pefntvty', 
                    'prcitshp', 
                    'prinuyer',
                    'prempnot',
                    'prftlf',
                    'prcow1',
                    'prmjind1',
                    'prmjocc1', 
                    'peschenr', 
                    'prnmchld',
                    'peafwhn1',
                    'pecert1',\
                    
        ]

In [101]:
df.columns = df.columns.str.lower()

In [102]:
df = df[columns_to_keep]

In [103]:
df = df.rename(columns={'hehousut': 'housing_type',
        'hrnumhou': 'household_num',
        'hefaminc': 'family_income',
        'hrhtype': 'household_type',
        'hubus': 'own_bus_or_farm',
        'gediv': 'region',
        'gestfips': 'state',
        'gtmetsta': 'metropolitan',
        'gtcbsasz': 'metro_area_size',
        'prtage': 'age',
        'pemaritl': 'marital_status',
        'pesex': 'is_male',
        'peafever': 'veteran',
        'peeduca': 'education',
        'ptdtrace': 'race',
        'pehspnon': 'hispanic_or_non',
        'penatvty': 'birth_country',
        'pemntvty': 'mother_birth_country',
        'pefntvty': 'father_birth_country',
        'prcitshp': 'citizenship',        
        'prinuyer': 'immigration_era',
        'prempnot': 'employed',
        'prftlf': 'ft_or_pt',
        'prcow1': 'worker_class',
        'prmjind1': 'industry',
        'prmjocc1': 'occupation',
        'peschenr': 'enrolled_in_school',
        'prnmchld': 'num_children',
        'peafwhn1': 'service_era',
        'pecert1': 'professional_certification'})

# Prepare (Values)

### drop any NILF(not in labor force or with -1)

In [104]:
df.employed.value_counts(0)

 1.0    48086
 4.0    33496
-1.0    17580
 2.0     1589
 3.0       88
Name: employed, dtype: int64

In [105]:
df = df[(df.employed == 1) | (df.employed == 2)].reset_index(drop=True)

### check for null values

In [106]:
df.isna()

,housing_type,household_num,family_income,household_type,own_bus_or_farm,country_region,state,metropolitan,metro_area_size,age,...,immigration_era,employed,ft_or_pt,worker_class,industry,occupation,enrolled_in_school,num_children,service_era,professional_certification
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49670,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
49671,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
49672,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
49673,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [107]:
df.household_num.value_counts()

2     16209
3      9881
4      9622
1      6358
5      4511
6      1905
7       692
8       278
9       129
10       36
11       35
12       12
13        7
Name: household_num, dtype: int64

### check for -1 values (those are the N/A type responses in the survey). Will try to impute them depeneding on the category

In [108]:
for col in df.columns:
    print(col)
    print(df[col].value_counts().sort_values())
    print('---')

housing_type
8         2
12       10
4        12
7        13
3        13
2        17
6       220
5      1556
1     47832
Name: housing_type, dtype: int64
---
household_num
13        7
12       12
11       35
10       36
9       129
8       278
7       692
6      1905
5      4511
1      6358
4      9622
3      9881
2     16209
Name: household_num, dtype: int64
---
family_income
3       246
2       285
5       384
1       426
4       466
6       725
7      1088
8      1251
9      1844
10     2084
11     3248
12     3748
13     5248
14     7342
15     9544
16    11746
Name: family_income, dtype: int64
---
household_type
10        4
9         8
8        16
5        17
2       189
3      3428
7      4805
4      6125
6      6162
1     28921
Name: household_type, dtype: int64
---
own_bus_or_farm
-1        1
-2       41
-3      119
 1     8518
 2    40996
Name: own_bus_or_farm, dtype: int64
---
country_region
6    3267
1    3976
2    4202
4    4825
3    5245
7    5470
8    6184
9    7377
5    

### takeaways
* columns to adjust
    own_bus_or_farm, veteran, unpaid_work_last_week, usual_35_or_more
* to drop, job_past_12_months, worker_class, industry, occupation, hourly, enrolled_in_school, ged_or_diploma, service_era
---

## imputing values and handling -1
* need to handle the -1 values by reading original data dictionary for guidance
* an answer not in the affirmative is treated as a negative for binary classification
    * own_bus_or_farm, veteran, service_era
* delete columns
    * unpaid, usual 35 or more, job past 12 months
* impute (decide on method) --> KNN impute
    * enrolled_in_school

In [109]:
df['own_bus_or_farm'] = df['own_bus_or_farm'].apply(lambda x: x == 1, 1,0)

In [110]:
df['veteran'] = df['veteran'].apply(lambda x: x == 1, 1,0)

In [111]:
df['service_era'] = df.service_era.apply(lambda x: 0 if x == -1 else x)

In [112]:
df.enrolled_in_school = df.enrolled_in_school.apply(lambda x: np.nan if x == -1 else x)

In [113]:
df[['worker_class', 'industry','occupation']] = df[['worker_class', 'industry','occupation']].applymap(lambda x: np.nan if x == -1 else x)

In [114]:
from sklearn.impute import KNNImputer

In [115]:
imputer = KNNImputer(n_neighbors=1)
df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

### Takeaways
* Nothing of major importance 
* was difficult understanding how to utilize the KNN imputer with the categorical nature of the column being imputed
---

## Flattening Columns

* Flattening is decreasing categorical optionality and renaming numerical entries
* Columns to flatten: 
 * housing_type, family income, household_type, marital_status, education, race, citzenship, birth country, mother birth country, father birth country, immigration era, service era

In [117]:
# create function to flatten race in less categories
def flatten_housing_type(val):
    """ 
    Purpose:
        To reduce the number of options within the feature
    ---
    Parameters:
        val
    ---
    Returns:
        val
    """
    if val == 1:
        val = 'perm_house_apt'
    elif 5 <= val <= 6:
        val = 'mobile_home'
    else:
        val = 'other'
    return val

In [118]:
def flatten_family_income(val):
    """ 
    Purpose:
       To reduce the number of options within the feature 
    ---
    Parameters:
        val
    ---
    Returns:
        val
    """

    if 1 <= val <=11:
        val = '0 TO 49,999'
    elif val in [12,13]:
        val = '50,000 to 74,999'
    elif val == 14:
        val = '75,000 to 99,999'
    elif val == 15:
        val = '100,000 to 149,999'
    else: 
        val = '150,000 or More'
    return val

In [119]:

def flatten_household_type(val):
    ''' 
    Purpose:
    To reduce the number of options  within the feature
    ---
    Parameters:
        val
    ---
    Output:
        val
    ---
    '''
    if val == 0:
        val = 'unknown'
    elif 1<= val <= 2:
        val = 'married'
    elif 3 <= val <= 8:
        val = 'unmarried'
    else:
        val = 'group'
    return val


In [120]:

def flatten_marital(val):
    ''' 
    Purpose:
    To reduce the number of options  within the feature
    ---
    Parameters:
        val
    ---
    Output:
        val
    ---
    '''
    if val in [1,2,5]:
        val = 'married'
    else:
        val = 'single'
    return val


In [121]:
def flatten_education(val):
    ''' 
    Purpose:
    To reduce the number of options  within the feature
    ---
    Parameters:
        val
    ---
    Output:
        val
    ---
    '''
    if val <= 38:
        val = 'no_high_school'
    elif val <= 40:
        val = 'high_school_ged'
    elif val < 43:
        val = 'associates'
    elif val == 43:
        val = 'bachelor'
    elif val > 43:
        val = 'post_grad'
    return val


In [122]:
# create function to flatten race in less categories
def flatten_race(val):
    ''' 
    Purpose:
        To reduce the number of options  within the feature
    ---
    Parameters:
        val
    ---
    Output:
        val
    ---
    '''
    if val == 1:
        val = 'white'
    elif val == 2:
        val = 'black'
    elif val == 3:
        val = 'AI/NA'
    elif val == 4:
        val = 'asian'
    elif val == 5:
        val = 'HI/PI'
    elif val in [6,7,8,9,16,17,18,19,20,21,23,24]:
        val = 'mixed_white'
    else :
        val = 'mixed_other'
    return val


In [123]:

def flatten_citizenship(val):
    ''' 
    Purpose:
    To reduce the number of options  within the feature
    ---
    Parameters:
        val
    ---
    Output:
        val
    ---
    '''
    if val <= 3:
        val = 'native'
    elif val == 4 :
        val = 'naturalized'
    else :
        val = 'foreign'

    return val


In [124]:

def flatten_immigration(val):
    ''' 
    Purpose:
    To reduce the number of options  within the feature
    ---
    Parameters:
        val
    ---
    Output:
        val
    ---
    '''
    if val == 57:
        val = 'us_50'
    elif val < 100:
        val = 'us_territories'
    else:
        val = 'foreign_country'
    return val


In [125]:
def flatten_immigration_era(val):
    ''' 
    Purpose:
    To reduce the number of options  within the feature
    ---
    Parameters:
        val
    ---
    Output:
        val
    ---
    '''
    if val <= 0:
        val = 'native'
    elif val < 17:
        val = 'pre_gwot'
    else:
        val = 'post_pwot'
    return val

In [126]:

def flatten_service_era(val):
    ''' 
    Purpose:
    To reduce the number of options  within the feature
    ---
    Parameters:
        val
    ---
    Output:
        val
    ---
    '''
    if val == 1:
        val = 'post_gwot'
    elif val > 1:
        val = 'pre_gwot'
    else:
        val = 'n/a'
    return val

In [127]:
#flattens feature
df.family_income = df.family_income.apply(flatten_family_income)

In [128]:
#flattens feature
df.housing_type = df.housing_type.apply(flatten_housing_type)

In [129]:
#flattens feature
df.household_type = df.household_type.apply(flatten_household_type)

In [130]:
#flattens feature
df.marital_status = df.marital_status.apply(flatten_marital)

In [131]:
#flattens feature
df['race'] = df['race'].apply(flatten_race)

In [132]:
#flattens feature
df.education = df.education.apply(flatten_education)

In [133]:
#flattens feature
df.citizenship = df.citizenship.apply(flatten_citizenship)

In [134]:
#flattens features
df.birth_country = df.birth_country.apply(flatten_immigration)
df.mother_birth_country = df.mother_birth_country.apply(flatten_immigration)
df.father_birth_country = df.father_birth_country.apply(flatten_immigration)

In [135]:
#flattens feature
df.immigration_era = df.immigration_era.apply(flatten_immigration_era)

In [136]:
#flattens feature
df.service_era = df.service_era.apply(flatten_service_era)

## Renaming Column Values 
* Columns with values to be renamed
    * region, state, metropolitan, ft_or_pt, worker_class, industry, occupation 
* Binary Columns 
    * own_bus_or_farm, is_male, veteran, hispanic_or_non, employed,  professional_certification, enrolled in schoool

In [137]:
#prints value counts of features for inspection
for col in df.columns:
    print(col)
    print(df[col].value_counts().sort_values())
    print('---')

housing_type
other                67
mobile_home        1776
perm_house_apt    47832
Name: housing_type, dtype: int64
---
household_num
13.0        7
12.0       12
11.0       35
10.0       36
9.0       129
8.0       278
7.0       692
6.0      1905
5.0      4511
1.0      6358
4.0      9622
3.0      9881
2.0     16209
Name: household_num, dtype: int64
---
family_income
75,000 to 99,999       7342
50,000 to 74,999       8996
100,000 to 149,999     9544
150,000 or More       11746
0 TO 49,999           12047
Name: family_income, dtype: int64
---
household_type
group           12
unmarried    20553
married      29110
Name: household_type, dtype: int64
---
own_bus_or_farm
1.0     8518
0.0    41157
Name: own_bus_or_farm, dtype: int64
---
country_region
6.0    3267
1.0    3976
2.0    4202
4.0    4825
3.0    5245
7.0    5470
8.0    6184
9.0    7377
5.0    9129
Name: country_region, dtype: int64
---
state
23.0     386
44.0     451
10.0     500
9.0      546
21.0     549
2.0      586
24.0     597


In [138]:
#map for adjusting numerical regions to their string counterparts
region_map = {1: 'NEW ENGLAND', 2: 'MIDDLE ATLANTIC', 3: 'EAST NORTH CENTRAL', 4: 'WEST NORTH CENTRAL',
            5: 'SOUTH ATLANTIC',6: 'EAST SOUTH CENTRAL',7: 'WEST SOUTH CENTRAL', 8: 'MOUNTAIN',  9: 'PACIFIC' }

In [ ]:
#map for adjusting numerical states to their string counterparts
state_map = {1:'Alabama',2:'Alaska',4:'Arizona',5:'Arkansas', 6:'California',8:'Colorado', 9:'Conneticut', 10:'Deleware',
            11:'D.C.', 12:'Florida',13:'Georgia',15:'Hawaii',16:'Idaho',17:'Illinois',18:'Indiana',
            19:'Iowa',20:'Kansas',21:'Kentucky',22:'Lousiana',23:'Maine',24:'Maryland',25:'Massachusets',26:'Michigan',
            27:'Minnesota',28:'Mississippi',29:'Missouri',30:'Montana',31:'Nebraska',32:'Nevada',33:'New Hampshire',34:'New Jersey',
            35:'New Mexico',36:'New York',37:'North Carolina',38:'North Dakota',39:'Ohio',40:'Oklahoma',41:'Oregon',42:'Pennyslyvania',
            44:'Rhode Island',45:'South Carolina',46:'South Dakota',47:'Tennesse',48:'Texas',49:'Utah',50:'Vermont',51:'Virginia',
            53:'Washignton',54:'West Virginia',55 :'Wisconsin',56:'Wyoming'}

In [139]:
# state_map = {1:'AL',2:'AK',4:'AZ',5:'AR', 6:'CA',8:'CO', 9:'CT', 10:'DE',
#             11:'DC', 12:'FL',13:'GA',15:'HI',16:'ID',17:'IL',18:'IN',
#             19:'IA',20:'KS',21:'KY',22:'LA',23:'ME',24:'MD',25:'MA',26:'MI',
#             27:'MN',28:'MS',29:'MO',30:'MT',31:'NE',32:'NV',33:'NH',34:'NJ',
#             35:'NM',36:'NY',37:'NC',38:'ND',39:'OH',40:'OK',41:'OR',42:'PA',
#             44:'RI',45:'SC',46:'SD',47:'TN',48:'TX',49:'UT',50:'VT',51:'VA',
#             53:'WA',54:'WV',55 :'WI',56:'WY'}

In [140]:
#map for ajusting metropolitan area
metropolitan_map = {1:'metro', 2:'non_metro',3:'not_identified'}

In [141]:
#map for full time for part time disposition
ft_pt_map = {1: 'full_time', 2: 'part_time'}

In [142]:
#map for worker class disposition
worker_class_map = {1:'FEDERAL GOVT', 2:'STATE GOVT', 3:'LOCAL GOVT',
                    4:'PRIVATE',5: 'SELF-EMPLOYED, UNINCORP.', 6: 'WITHOUT PAY'}

In [143]:
#map for adjusting numerical indstury labels
industry_map = {1:'Agriculture, forestry, fishing, and hunting',2:'Mining',3:'Construction',										 
				4:'Manufacturing', 5:'Wholesale and retail trade', 6:'Transportation and utilities',					      
				7:'Information', 8:'Financial activities', 9:'Professional and business services',					
				10:	'Educational and health services', 11:'Leisure and hospitality',12:'Other services',									
				13:'Public administration', 14:'Armed Forces'}

In [144]:
#map for adjusting numerical occupation labels
occupation_map = {1:'Management, business, and financial occupations',
                2:	'Professional and related occupations',					
                3:	'Service occupations',				
                4:	'Sales and related occupations',					
                5:	'Office and administrative support occupations',
                6:	'Farming, fishing, and forestry occupations',			
                7:	'Construction and extraction occupations',				
                8:	'Installation, maintenance, and repair occupations',	
                9:	'Production occupations',						
                10:	'Transportation and material moving occupations',	
                11:	'Armed Forces'}

In [ ]:
### applies mappings
df.region = df.region.map(region_map)
df.state = df.state.map(state_map)
df.metropolitan = df.metropolitan.map(metropolitan_map)
df.ft_or_pt = df.ft_or_pt.map(ft_pt_map)
df.worker_class = df.worker_class.map(worker_class_map)
df.industry = df.industry.map(industry_map)
df.occupation = df.occupation.map(occupation_map)

## Categorical vs Numerical
* create categorical dtypes for proper column orderning
    * family income, education, metropolitan size

In [153]:
#fixing types on categorical columns
categorical_cols = ['housing_type','family_income','household_type',
                    'region','state','metropolitan','metro_area_size',
                    'marital_status','education','race','birth_country',
                    'mother_birth_country','father_birth_country','citizenship',
                    'immigration_era', 'ft_or_pt', 'worker_class',
                    'industry','occupation', 'service_era']

#for loop to handle assignment as object
for col in categorical_cols:
    df[col] = (df[col].astype('category'))

In [154]:
# correcting sort ordinality
from pandas.api.types import CategoricalDtype

In [155]:
df.family_income.value_counts()

0 TO 49,999           12047
150,000 or More       11746
100,000 to 149,999     9544
50,000 to 74,999       8996
75,000 to 99,999       7342
Name: family_income, dtype: int64

In [156]:
#creates categorical dtype for later ordering of family income column
income_order = CategoricalDtype(['0 TO 49,999', '50,000 to 74,999', '75,000 to 99,999','100,000 to 149,999','150,000 or More'], ordered=True)

In [157]:
df.family_income = df.family_income.astype(income_order)

In [158]:
df.education.value_counts().index

CategoricalIndex(['high_school_ged', 'bachelor', 'post_grad', 'associates',
                  'no_high_school'],
                 categories=['associates', 'bachelor', 'high_school_ged', 'no_high_school', 'post_grad'], ordered=False, dtype='category')

In [159]:
#creates categorical dtype for education column ordering
education_order = CategoricalDtype(['no_high_school','high_school_ged','associates','bachelor','post_grad'], ordered=True)

In [160]:
df.education = df.education.astype(education_order)

In [161]:
def prep_column_order(df):
    """ 
    Purpose:
       to apply created custom column ordering to specified columns 
    ---
    Parameters:
        df: a dataframe containing the october cps
    ---
    Returns:
        df: a dataframe with the adjusted columns
    """
    income_order = CategoricalDtype(['0 TO 49,999', '50,000 to 74,999', '75,000 to 99,999','100,000 to 149,999','150,000 or More'], ordered=True)
    education_order = CategoricalDtype(['no_high_school','high_school_ged','associates','bachelor','post_grad'], ordered=True)

    df.family_income = df.family_income.astype(income_order)
    df.education = df.education.astype(education_order)

    return df

In [162]:
#fixing binary columns freom -1, 1, 2 --> 0, 1
binary_cols = ['own_bus_or_farm', 'is_male', 'veteran','hispanic_or_non', 'employed',
            'enrolled_in_school','professional_certification']


In [163]:
# changed all non-affirmative answers to negative. school, most affected
for col in binary_cols:
    df[col] = np.where(df[col] == 1, 1, 0)

In [164]:
for col in df.columns:
    print(col)
    print(df[col].value_counts().sort_values())
    print('---')

housing_type
other                67
mobile_home        1776
perm_house_apt    47832
Name: housing_type, dtype: int64
---
household_num
13.0        7
12.0       12
11.0       35
10.0       36
9.0       129
8.0       278
7.0       692
6.0      1905
5.0      4511
1.0      6358
4.0      9622
3.0      9881
2.0     16209
Name: household_num, dtype: int64
---
family_income
75,000 to 99,999       7342
50,000 to 74,999       8996
100,000 to 149,999     9544
150,000 or More       11746
0 TO 49,999           12047
Name: family_income, dtype: int64
---
household_type
group           12
unmarried    20553
married      29110
Name: household_type, dtype: int64
---
own_bus_or_farm
1     8518
0    41157
Name: own_bus_or_farm, dtype: int64
---
country_region
EAST SOUTH CENTRAL    3267
NEW ENGLAND           3976
MIDDLE ATLANTIC       4202
WEST NORTH CENTRAL    4825
EAST NORTH CENTRAL    5245
WEST SOUTH CENTRAL    5470
MOUNTAIN              6184
PACIFIC               7377
SOUTH ATLANTIC        9129
Name:

# Prep: Functions for output to wrangle.py
* Goal:
    - Create functions that will handle sections of prep 
    - Big wrangle Function --> wrangle_oct()
    - don't forget to add docstring placeholders

In [184]:
def acquire_oct():
    ''' 
    Purpose:
        Initial acquisition of October data
    ---
    Parameters:
        None
    ---
    Output:
        df: dataframe containing the data with column headers in lowercase
    ---
    '''

    df = pd.read_csv('oct22pub.csv')

    df.columns = df.columns.str.lower()
    
    return df


def prep_columns(df): 
    """ 
    Purpose:
        to filter and rename columns for initial dataframe
    ---
    Parameters:
        df: dataframe
    ---
    Returns:
        df: adjusted dataframe
    """
    #creates mask to be used for filtering
    columns_to_keep = [ 'hehousut',
                        'hrnumhou', 
                        'hefaminc',
                        'hrhtype',
                        'hubus',
                        'gediv',
                        'gestfips',
                        'gtmetsta',
                        'gtcbsasz',
                        'prtage',
                        'pemaritl',
                        'pesex',
                        'peafever',
                        'peeduca',
                        'ptdtrace', 
                        'pehspnon',
                        'penatvty',
                        'pemntvty',
                        'pefntvty', 
                        'prcitshp', 
                        'prinuyer',
                        'prempnot',
                        'prftlf',
                        'prcow1',
                        'prmjind1',
                        'prmjocc1', 
                        'peschenr', 
                        'prnmchld',
                        'peafwhn1',
                        'pecert1',\
                        
            ]

    #filters columns using mask
    df = df[columns_to_keep]

    #renames columns with appropriate labels that are adapting from official naming convention
    df = df.rename(columns={'hehousut': 'housing_type',
        'hrnumhou': 'household_num',
        'hefaminc': 'family_income',
        'hrhtype': 'household_type',
        'hubus': 'own_bus_or_farm',
        'gediv': 'region',
        'gestfips': 'state',
        'gtmetsta': 'metropolitan',
        'gtcbsasz': 'metro_area_size',
        'prtage': 'age',
        'pemaritl': 'marital_status',
        'pesex': 'is_male',
        'peafever': 'veteran',
        'peeduca': 'education',
        'ptdtrace': 'race',
        'pehspnon': 'hispanic_or_non',
        'penatvty': 'birth_country',
        'pemntvty': 'mother_birth_country',
        'pefntvty': 'father_birth_country',
        'prcitshp': 'citizenship',        
        'prinuyer': 'immigration_era',
        'prempnot': 'employed',
        'prftlf': 'ft_or_pt',
        'prcow1': 'worker_class',
        'prmjind1': 'industry',
        'prmjocc1': 'occupation',
        'peschenr': 'enrolled_in_school',
        'prnmchld': 'num_children',
        'peafwhn1': 'service_era',
        'pecert1': 'professional_certification'})                               

    return df

def generate_mappings():
    """ 
    Purpose:
        to generate feature mappings to adjust away from numerical labeling
    ---
    Parameters:
        none
    ---
    Returns:
        region_map: map for adjusting numerical regions to their string counterparts
        state_map : map for adjusting numerical states to their string counterparts
        metropolitan_map : map for ajusting metropolitan area
        ft_pt_map: map for full time for part time disposition
        worker_class_map : map for worker class disposition
        industry_map: map for adjusting numerical indstury labels
        occupation_map: map for adjusting numerical occupation labels
    """
    region_map = {1: 'NEW ENGLAND', 2: 'MIDDLE ATLANTIC', 3: 'EAST NORTH CENTRAL', 4: 'WEST NORTH CENTRAL',
            5: 'SOUTH ATLANTIC',6: 'EAST SOUTH CENTRAL',7: 'WEST SOUTH CENTRAL', 8: 'MOUNTAIN',  9: 'PACIFIC' }

    state_map = {1:'AL',2:'AK',4:'AZ',5:'AR', 6:'CA',8:'CO', 9:'CT', 10:'DE',
                11:'DC', 12:'FL',13:'GA',15:'HI',16:'ID',17:'IL',18:'IN',
                19:'IA',20:'KS',21:'KY',22:'LA',23:'ME',24:'MD',25:'MA',26:'MI',
                27:'MN',28:'MS',29:'MO',30:'MT',31:'NE',32:'NV',33:'NH',34:'NJ',
                35:'NM',36:'NY',37:'NC',38:'ND',39:'OH',40:'OK',41:'OR',42:'PA',
                44:'RI',45:'SC',46:'SD',47:'TN',48:'TX',49:'UT',50:'VT',51:'VA',
                53:'WA',54:'WV',55 :'WI',56:'WY'}

    metropolitan_map = {1:'metro', 2:'non_metro',3:'not_identified'}

    ft_pt_map = {1: 'full_time', 2: 'part_time'}

    worker_class_map = {1:'FEDERAL GOVT', 2:'STATE GOVT', 3:'LOCAL GOVT',
                        4:'PRIVATE',5: 'SELF-EMPLOYED, UNINCORP.', 6: 'WITHOUT PAY'}

    industry_map = {1:'Agriculture, forestry, fishing, and hunting',2:	'Mining',3:	'Construction',										 
                    4:	'Manufacturing', 5:	'Wholesale and retail trade', 6:'Transportation and utilities',					      
                    7:	'Information', 8:'Financial activities', 9:'Professional and business services',					
                    10:	'Educational and health services', 11:'Leisure and hospitality',12:'Other services',									
                    13:'Public administration', 14:'Armed Forces'}

    occupation_map = {1:'Management, business, and financial occupations',
                    2:	'Professional and related occupations',					
                    3:	'Service occupations',				
                    4:	'Sales and related occupations',					
                    5:	'Office and administrative support occupations',
                    6:	'Farming, fishing, and forestry occupations',			
                    7:	'Construction and extraction occupations',				
                    8:	'Installation, maintenance, and repair occupations',	
                    9:	'Production occupations',						
                    10:	'Transportation and material moving occupations',	
                    11:	'Armed Forces'}

    return region_map, state_map, metropolitan_map, ft_pt_map, worker_class_map, industry_map, occupation_map

def prep_values(df):
    ''' 
    Purpose:
        to prepare dataframe values for exploration and modeling
    ---

    Parameters:
        df: a dataframe
    ---

    Output:
        df: a dataframe
    ---
    '''

    #filters employed/unemployed from those not in labor force or without response
    df = df[(df.employed == 1) | (df.employed == 2)].reset_index(drop=True)

    ### imputing values ###
    df['own_bus_or_farm'] = df['own_bus_or_farm'].apply(lambda x: x == 1, 1,0)
    df['veteran'] = df['veteran'].apply(lambda x: x == 1, 1,0)
    df['service_era'] = df.service_era.apply(lambda x: 0 if x == -1 else x)
    df.enrolled_in_school = df.enrolled_in_school.apply(lambda x: np.nan if x == -1 else x)
    df[['worker_class', 'industry','occupation']] = df[['worker_class', 'industry','occupation']].applymap(lambda x: np.nan if x == -1 else x)

    imputer = KNNImputer(n_neighbors=1)
    df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

    ### flatten values ###
    df.housing_type = df.housing_type.apply(flatten_housing_type)
    df.family_income.apply(flatten_family_income).value_counts()
    df.household_type = df.household_type.apply(flatten_household_type)
    df.marital_status = df.marital_status.apply(flatten_marital)
    df['race'] = df['race'].apply(flatten_race)
    df.education = df.education.apply(flatten_education)
    df.citizenship = df.citizenship.apply(flatten_citizenship)
    df.birth_country = df.birth_country.apply(flatten_immigration)
    df.mother_birth_country = df.mother_birth_country.apply(flatten_immigration)
    df.father_birth_country = df.father_birth_country.apply(flatten_immigration)
    df.immigration_era = df.immigration_era.apply(flatten_immigration_era)
    df.service_era = df.service_era.apply(flatten_service_era)

    ### generate and use value mappings ###
    (region_map, state_map, metropolitan_map, ft_pt_map,
     worker_class_map, industry_map, occupation_map) = generate_mappings()

    df.region = df.region.map(region_map)
    df.state = df.state.map(state_map)
    df.metropolitan = df.metropolitan.map(metropolitan_map)
    df.ft_or_pt = df.ft_or_pt.map(ft_pt_map)
    df.worker_class = df.worker_class.map(worker_class_map)
    df.industry = df.industry.map(industry_map)
    df.occupation = df.occupation.map(occupation_map)

    #fixing binary columns freom -1, 1, 2 --> 0, 1
    binary_cols = ['own_bus_or_farm', 'is_male', 'veteran','hispanic_or_non', 'employed',
                'enrolled_in_school','professional_certification']

    # changed all non-affirmative answers to negative. school, most affected
    for col in binary_cols:
        df[col] = np.where(df[col] == 1, 1, 0)

    return df

def prep_column_order(df):
    """ 
    Purpose:
       to apply created custom column ordering to specified columns 
    ---
    Parameters:
        df: a dataframe containing the october cps
    ---
    Returns:
        df: a dataframe with the adjusted columns
    """
    income_order = CategoricalDtype(['0 TO 49,999', '50,000 to 74,999', '75,000 to 99,999','100,000 to 149,999','150,000 or More'], ordered=True)
    education_order = CategoricalDtype(['no_high_school','high_school_ged','associates','bachelor','post_grad'], ordered=True)

    df.family_income = df.family_income.astype(income_order)
    df.education = df.education.astype(education_order)
    return df

In [177]:
def scale_data(df):
    """ 
    Purpose:
        to scale dataframe data
    ---
    Parameters:
        df: a dataframe
    ---
    Returns:
        df: a dataframe
    """
    #initializes and uses scaler on appropirate numerical columns
    scaler = MinMaxScaler()
    to_scale = ['household_num', 'num_children']
    scaler.fit(df[to_scale])

    df[to_scale]=scaler.transform(df[to_scale])

    return df

In [178]:
df_scaled = scale_data(df)

In [179]:
from sklearn.preprocessing import OneHotEncoder

In [180]:
def split_data(df):
    ''' 
    Purpose:
        To split the input dataframe
    ---
    Parameters:
        df: a tidy dataframe
    ---
    Output:
        train: subset of dataframe for model training
        validate: unseen data for model testing
        test: unseen data for final model test
    ---
    '''
    #train_test_split
    train_validate, test = train_test_split(df, test_size=.2, random_state=514, stratify=df['employed'])
    train, validate = train_test_split(train_validate, test_size=.3, random_state=514, stratify=train_validate['employed'])

    return train, validate, test

In [181]:
train, validate, test = split_data(df_scaled)

In [185]:
def wrangle_oct(explore=False, model=False):
    ''' 
    Purpose:
        to wrangle data for all phases of pipeline
    ---

    Parameters:
        explore: informs function whether the appropriate phase is explore or not
        model: informs function whether the apprpriate phase is explore of not
    ---

    Output:
        train: the train subset of data, if explore parameter is True, this is only subset returned
        train_scaled: scaled version of train subset used for modeling
        validate: scaled version of validate subset used for modeling
        test: scaled version of test subset used for modeling
    ---
    '''

    df = acquire_oct()

    df = prep_columns(df)

    df = prep_values(df)

    df = prep_column_order(df)

    if model:
        df = df.drop(columns=['state', 'birth_country','mother_birth_country',
                                'father_birth_country','citizenship', 'immigration_era'])

    train, validate, test = split_data(df)

    if explore:
        return train
    
    #gets dummies of data
    train = pd.get_dummies(train)
    validate = pd.get_dummies(validate)
    test = pd.get_dummies(test)

    train_scaled = scale_data(train)  
    validate = scale_data(validate)
    test = scale_data(test)

    return train, train_scaled, validate, test

In [195]:
train, train_scaled, validate, test = wrangle_oct(model=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49675 entries, 0 to 49674
Data columns (total 30 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   housing_type                49675 non-null  object 
 1   household_num               49675 non-null  float64
 2   family_income               49675 non-null  float64
 3   household_type              49675 non-null  object 
 4   own_bus_or_farm             49675 non-null  int64  
 5   country_region              49675 non-null  object 
 6   state                       49675 non-null  object 
 7   metropolitan                49675 non-null  object 
 8   metro_area_size             49675 non-null  float64
 9   age                         49675 non-null  float64
 10  marital_status              49675 non-null  object 
 11  is_male                     49675 non-null  int64  
 12  veteran                     49675 non-null  int64  
 13  education                   496

In [ ]:
def split_X_y(train, validate, test):
    ''' 
    Purpose:
    ---
        To split the subsets further for modeling and testing
    ---
    Parameters:
        train: unscaled subset of dataframe for exploration and model training
        validate: unscaled and unseen data for model testing
        test: unscaled and unseen data for final model test
    ---
    Output:
        X_train: features to fit model and make predictions
        y_train: target variable outcome for model evaluation
        X_validate: features to make predictions
        y_validate: target variable outcome for model evaluation
        X_test: features to make predictions
        y_test: target variable outcome for model evaluation
    ---
    '''
    # split data into Big X, small y sets 
    X_train = train.drop(columns=['employed'])
    y_train = train.employed

    X_validate = validate.drop(columns=['employed'])
    y_validate = validate.employed

    X_test = test.drop(columns=['employed'])
    y_test = test.employed

    return X_train, y_train, X_validate, y_validate, X_test, y_test

In [ ]:
def model_prep():
    """ 
    Purpose:
        to combine wrangling and splitting funcations for modeling phase
    ---
    Parameters:
        none
    ---
    Returns:
        X_train, y_train, X_validate, y_validate, X_test, y_test: subsets for modeling
    """

    train, train_scaled, validate, test = wrangle_oct()

    (X_train, y_train, X_validate,
     y_validate, X_test, y_test) = split_X_y(train_scaled, validate, test)

    return X_train, y_train, X_validate, y_validate, X_test, y_test

In [ ]:
X_train, y_train, X_validate, y_validate, X_test, y_test = model_prep()

In [ ]:
reload(wrangle)

<module 'wrangle' from '/Users/sinao/coding_projects/oct_2022_unemployment_project/wrangle.py'>

In [ ]:
train, X_train, y_train, X_validate, y_validate, X_test, y_test = wrangle.model_prep()

In [ ]:
train.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 27818 entries, 3549 to 30192
Columns: 150 entries, household_num to service_era_pre_gwot
dtypes: float64(4), int64(7), uint8(139)
memory usage: 6.2 MB
